[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/03_Audit_Trail_and_Report.ipynb)


In [1]:
import os
import sys
from pathlib import Path
import subprocess

def _ensure_repo():
    workspace = os.environ.get('GITHUB_WORKSPACE')
    if workspace:
        repo_root = Path(workspace).resolve()
        src_path = repo_root / 'src'
        if src_path.exists():
            sys.path.insert(0, str(src_path))
            return repo_root
    repo_root = Path().resolve()
    src_path = repo_root / 'src'
    if src_path.exists():
        sys.path.insert(0, str(src_path))
        return repo_root
    repo_dir = Path('IINTS-SDK')
    if not repo_dir.exists():
        subprocess.check_call(['git', 'clone', 'https://github.com/python35/IINTS-SDK.git'])
    repo_root = repo_dir.resolve()
    os.chdir(repo_root)
    src_path = repo_root / 'src'
    sys.path.insert(0, str(src_path))
    return repo_root

_ensure_repo()


PosixPath('/home/runner/work/IINTS-SDK/IINTS-SDK')

# Audit Trail and Clinical Report

This notebook shows how to export the audit log and generate a PDF report.


In [2]:
import json
from iints.presets import load_presets

import iints
from iints.core.algorithms.pid_controller import PIDController

presets = load_presets()
preset = next(p for p in presets if p["name"] == "baseline_t1d")

outputs = iints.run_simulation(
    algorithm=PIDController,
    scenario=preset["scenario"],
    patient_config=preset["patient_config"],
    duration_minutes=240,
    time_step=preset["time_step_minutes"],
    seed=11,
    output_dir="results/audit_report",
    export_audit=True,
    generate_report=True,
)

outputs["audit"], outputs["report_pdf"]


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


({'jsonl': 'results/audit_report/audit/audit_trail.jsonl',
  'csv': 'results/audit_report/audit/audit_trail.csv',
  'summary': 'results/audit_report/audit/audit_summary.json'},
 'results/audit_report/clinical_report.pdf')

In [3]:
from pathlib import Path
import json

summary_path = Path(outputs["audit"]["summary"])
json.loads(summary_path.read_text())


{'total_steps': 13,
 'total_overrides': 13,
 'top_reasons': {'HYPO_CUTOFF: Glucose below safety cutoff; EMERGENCY_STOP: Severe hypoglycemia detected': 5,
  'WINDOW_CAP_EXCEEDED: 60min cap 3.00U reached': 1,
  'NEGATIVE_TREND_LIMIT: Glucose dropping at -4.27 mg/dL/min': 1,
  'NEGATIVE_TREND_LIMIT: Glucose dropping at -3.98 mg/dL/min': 1,
  'NEGATIVE_TREND_LIMIT: Glucose dropping at -3.45 mg/dL/min': 1,
  'NEGATIVE_TREND_LIMIT: Glucose dropping at -3.00 mg/dL/min': 1,
  'HYPO_CUTOFF: Glucose below safety cutoff; CRITICAL: Hypoglycemia - insulin limited; NEGATIVE_TREND_LIMIT: Glucose dropping at -2.62 mg/dL/min': 1,
  'HYPO_CUTOFF: Glucose below safety cutoff; EMERGENCY_STOP: Severe hypoglycemia detected; NEGATIVE_TREND_LIMIT: Glucose dropping at -2.29 mg/dL/min': 1,
  'HYPO_CUTOFF: Glucose below safety cutoff; EMERGENCY_STOP: Severe hypoglycemia detected; NEGATIVE_TREND_LIMIT: Glucose dropping at -2.01 mg/dL/min': 1},
 'terminated_early': True,
 'termination_reason': {'reason': 'Critical